In [ ]:
import time
import requests
import pandas as pd
from datetime import datetime, date, time as dt_time, timedelta
from zoneinfo import ZoneInfo
import time
from IPython.display import clear_output
import requests
import pandas as pd
import pandas_ta as ta
try:
    import talib
    USE_TALIB = True
except ImportError:
    USE_TALIB = False
from datetime import datetime, date, time as dt_time, timedelta
from zoneinfo import ZoneInfo
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# ── CONFIG ─────────────────────────────────────────────────────────────────
API_KEY = "vBRy5un9PuHfxFj1IrHpfg8a2RS57jE9"

# !pip install websocket-client
import websocket
print(websocket.__version__)

# https://github.com/polygon-io/client-python/blob/master/examples/websocket/stocks-ws.py

1.8.0


In [17]:
import websocket
import json
import threading
import pandas as pd
import time
from datetime import datetime
import talib
import os

API_KEY = "your_polygon_api_key"
TICKERS = ["TSLA", "AAPL", "MSFT"]
channels = ",".join([f"A.{ticker}" for ticker in TICKERS])  # Only 1m bars

# Initialize empty DataFrames for each ticker
live_dataframes = {ticker: pd.DataFrame(columns=["t", "o", "h", "l", "c", "v"]) for ticker in TICKERS}

# Directory to save CSV files
SAVE_DIR = "live_data"
os.makedirs(SAVE_DIR, exist_ok=True)

def on_open(ws):
    print("✅ Connected to Polygon WebSocket")
    ws.send(json.dumps({"action": "auth", "params": API_KEY}))
    ws.send(json.dumps({"action": "subscribe", "params": channels}))
    print(f"📡 Subscribed to: {channels}")

def on_message(ws, message):
    print("📨 RAW:", message)
    data = json.loads(message)

    for bar in data:
        if bar['ev'] == 'A':
            ticker = bar['sym']
            new_row = {
                "t": datetime.utcfromtimestamp(bar["s"] / 1000),  # Start time
                "o": bar["o"],
                "h": bar["h"],
                "l": bar["l"],
                "c": bar["c"],
                "v": bar["v"]
            }

            df = live_dataframes[ticker]
            if df.empty or new_row["t"] != df.iloc[-1]["t"]:
                live_dataframes[ticker] = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)

                # ⏺ Save new row to CSV
                csv_path = os.path.join(SAVE_DIR, f"{ticker}_live.csv")
                pd.DataFrame([new_row]).to_csv(csv_path, mode='a', header=not os.path.exists(csv_path), index=False)

                # 🧠 Run TA-lib and detect signals
                check_signals(ticker)

def on_error(ws, error):
    print("❌ Error:", error)

def on_close(ws, code, msg):
    print(f"🔌 Disconnected — Code: {code}, Msg: {msg}")

def run_websocket():
    socket = websocket.WebSocketApp(
        "wss://socket.polygon.io/stocks",
        on_open=on_open,
        on_message=on_message,
        on_error=on_error,
        on_close=on_close
    )
    socket.run_forever()

def check_signals(ticker):
    df = live_dataframes[ticker]
    if len(df) < 50:
        return  # not enough data yet for indicators

    # df = df.copy()

    # # Apply indicators
    # df["EMA9"] = talib.EMA(df["c"].values, timeperiod=9)
    # macd, signal, hist = talib.MACD(df["c"].values, fastperiod=12, slowperiod=26, signalperiod=9)
    # df["MACD"], df["Signal"], df["MACD_hist"] = macd, signal, hist
    # df["SMA5"] = talib.SMA(df["c"].values, timeperiod=5)
    # df["RSI"] = talib.RSI(df["c"].values, timeperiod=14)

    # df["RSI_smooth"] = df["RSI"].ewm(span=3, adjust=False).mean()
    # df["B_1"] = df["c"] >= df["SMA5"]
    # df["MACD_diff"] = df["MACD"] - df["Signal"]
    # df["MACD_diff_avg"] = df["MACD_diff"].rolling(15).mean()
    # df["MACD_strong"] = df["MACD_diff"] > 2 * df["MACD_diff_avg"]
    # df["B_2"] = (df["MACD_hist"].diff() > 0) & df["MACD_strong"]
    # df["B_3"] = df["RSI"] > df["RSI_smooth"]
    # df["Vol_avg20"] = df["v"].rolling(20).mean()
    # df["B_4"] = df["v"] > 1.5 * df["Vol_avg20"]
    # df["Cond_MACD_hist_avoid"] = (df["MACD_hist"] > 0) & (df["MACD_hist"].diff() < 0)
    # df["Cond_RSI_falling"] = df["RSI"].diff() < 0
    # df["Cond_price_lower"] = df["c"] < df["c"].shift(1)
    # df["B_5"] = df[["Cond_MACD_hist_avoid", "Cond_RSI_falling", "Cond_price_lower"]].any(axis=1)
    # N = 10
    # df["RecentHigh"] = df["h"].shift(1).rolling(N).max()
    # df["Breakout_follow_through"] = (df["h"] > df["RecentHigh"]) & (df["c"] > df["RecentHigh"]) & (df["c"] > df["o"])
    # df["B_6"] = df["Breakout_follow_through"]
    # df["Entry_Signal"] = df["B_1"] & df["B_2"] & df["B_3"] & ~df["B_5"] & df["B_6"]

    # # If latest row has signal:
    # if df.iloc[-1]["Entry_Signal"]:
    #     print(f"🚀 ENTRY SIGNAL on {ticker} at {df.iloc[-1]['t']} — Price: {df.iloc[-1]['c']}")

# Start WebSocket
thread = threading.Thread(target=run_websocket)
thread.start()

# Keep alive
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("🛑 Stopped by user")


✅ Connected to Polygon WebSocket
📡 Subscribed to: A.TSLA,A.AAPL,A.MSFT
📨 RAW: [{"ev":"status","status":"connected","message":"Connected Successfully"}]
📨 RAW: [{"ev":"status","status":"auth_failed","message":"authentication failed"}]
🔌 Disconnected — Code: 1000, Msg: 
🛑 Stopped by user


In [16]:
live_dataframes["TSLA"].tail()


,t,o,h,l,c,v


In [24]:
from polygon import WebSocketClient
from polygon.websocket.models import WebSocketMessage
from typing import List

# client = WebSocketClient("XXXXXX") # hardcoded api_key is used
client = WebSocketClient(API_KEY)  # POLYGON_API_KEY environment variable is used

# docs
# https://polygon.io/docs/stocks/ws_stocks_am
# https://polygon-api-client.readthedocs.io/en/latest/WebSocket.html#

# aggregates (per minute)
# client.subscribe("AM.*") # all aggregates
client.subscribe("AM.TSLA") # single ticker

# aggregates (per second)
# client.subscribe("A.*")  # all aggregates
# client.subscribe("A.TSLA") # single ticker

# trades
# client.subscribe("T.*")  # all trades
client.subscribe("T.TSLA") # multiple tickers

# quotes
# client.subscribe("Q.*")  # all quotes
client.subscribe("Q.TSLA") # multiple tickers


def handle_msg(msgs: List[WebSocketMessage]):
    for m in msgs:
        print(m)


# print messages
client.run(handle_msg)

AuthError: authentication failed

In [19]:
!pip install nest_asyncio
